In [1]:
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD, SSN, SOSA #most common namespaces
import urllib.parse #for parsing strings to URI's
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
# import stardog

In [2]:
df = pd.read_csv('Jasmin.csv')

In [3]:
df['Altitude']= pd.to_numeric(df['Altitude'],errors='coerce')
df['Altitude'].mean()

143.75714285714284

In [4]:
df.head(10)

,ID,LocalDate,LocalTime,GMTDate,GMTTime,lat,long,Temperature,Speed,Direction,Altitude,Cov,HDOP,Distance,Count
0,1SAT32,2011-10-26,07:40:35 AM,2011-10-25,11:40:35 PM,2.437206,185.111301,24.5,0.24,0,NaN,0,1.64,0,1
1,2SAT32,2011-10-26,03:41:35 PM,2011-10-26,07:41:35 AM,2.433402,185.108162,29.0,0.76,0,NaN,0,1.74,548,2
2,3SAT32,2011-10-26,11:40:35 PM,2011-10-26,03:40:35 PM,2.433734,185.108599,27.0,0.86,0,NaN,0,2.18,61,3
3,4SAT32,2011-10-27,11:40:34 PM,2011-10-27,03:40:34 PM,2.455738,185.183640,28.0,0.16,0,NaN,0,1.89,8669,4
4,5SAT32,2011-10-28,07:42:53 AM,2011-10-27,11:42:53 PM,2.456503,185.187684,27.5,1.46,0,NaN,1,2.50,456,5
5,6SAT32,2011-10-28,03:41:05 PM,2011-10-28,07:41:05 AM,2.483344,185.232541,33.5,0.06,0,NaN,5,1.45,5800,6
6,7SAT32,2011-10-28,03:41:05 PM,2011-10-28,07:41:05 AM,2.483344,185.232541,33.5,0.06,0,NaN,5,1.45,0,7
7,8SAT32,2011-10-28,03:41:05 PM,2011-10-28,07:41:05 AM,2.483344,185.232541,33.5,0.06,0,NaN,5,1.45,0,8
8,9SAT32,2011-10-28,07:40:34 PM,2011-10-28,11:40:34 AM,2.479701,185.243606,29.0,0.14,0,NaN,4,0.95,1291,9
9,10SAT32,2011-10-28,07:40:34 PM,2011-10-28,11:40:34 AM,2.479701,185.243606,29.0,0.14,0,NaN,5,0.95,0,10


In [5]:
df.describe()

,lat,long,Temperature,Speed,Direction,Altitude,Cov,HDOP,Distance,Count
count,899.000000,899.000000,899.000000,899.000000,899.000000,7.000000,899.000000,899.000000,899.000000,899.000000
mean,2.509015,183.746772,28.888765,0.587164,1.329255,143.757143,2.497219,2.550044,640.407119,450.000000
std,0.073845,9.925183,4.016236,0.673300,17.429289,88.443143,2.170552,3.812480,917.534269,259.663243
min,2.392483,117.981100,-35.500000,0.000000,0.000000,101.500000,0.000000,0.710000,0.000000,1.000000
25%,2.465016,185.199810,28.000000,0.180000,0.000000,103.050000,0.000000,1.295000,145.500000,225.500000
50%,2.490733,185.248029,28.500000,0.370000,0.000000,114.900000,2.000000,1.830000,341.000000,450.000000
75%,2.546202,185.311595,30.000000,0.735000,0.000000,120.200000,5.000000,2.715000,761.000000,674.500000
max,2.706847,185.403456,37.500000,6.610000,343.000000,343.400000,5.000000,99.900000,9603.000000,899.000000


In [6]:
g = Graph()
owl = Namespace('http://www.w3.org/2002/07/owl#')
sosa = Namespace('http://www.w3.org/ns/sosa/')
foo = Namespace('http://www.ontology/ns/foo/1.1#')
wgs_84 = Namespace('http://www.w3.org/2003/01/geo/wgs84_pos#')
time = Namespace('http://www.w3.org/2006/time#')
schema = Namespace('http://schema.org/')
XSD=Namespace('http://www.w3.org/2001/XMLSchema#')
Class = Namespace('http://purl.org/ontology/wo/Class/')
Family = Namespace('http://purl.org/ontology/wo/Family/')
Kingdom = Namespace('http://purl.org/ontology/wo/kingdom/')
Order = Namespace('http://purl.org/ontology/wo/Proboscidea/')
Phylum = Namespace('http://purl.org/ontology/wo/Phylum/')

In [7]:
for index, row in df.iterrows():
    
    g.add((URIRef(foo+row['ID']), RDF.type, sosa.Observation))
    g.add((URIRef(foo+row['ID']), sosa.hasResult, Literal(row['ID'], datatype=XSD.string))) 

    g.add((URIRef(foo+row['ID']), sosa.madeBySensor, URIRef(schema+'DGFC/elephant#Jasmin')))
    g.add((URIRef(foo+row['ID']), URIRef(schema+'dgfc/elephant#Jasmin'), Literal(row['ID'], datatype=XSD.string))) 
          
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'LocalDate')))
    g.add((URIRef(foo+'LocalDate'), sosa.resultTime, Literal(row['LocalDate'], datatype=XSD.date)))
    
    g.add((URIRef(foo+row['ID']),  sosa.observedProperty, URIRef(foo+'LocalTime')))
    g.add((URIRef(foo+'LocalTime'),  sosa.resultTime, Literal(row['LocalTime'], datatype=XSD.time)))
      
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'GMTDate')))
    g.add((URIRef(foo+'GMTDate'), sosa.resultTime, Literal(row['GMTDate'], datatype=XSD.date)))

    g.add((URIRef(foo+'GMTTime'), sosa.observedProperty, URIRef(foo+'GMTTime')))
    g.add((URIRef(foo+'GMTTime'), sosa.resultTime,  Literal(row['GMTTime'], datatype=XSD.time)))

    g.add((URIRef(foo+row['ID']),  sosa.observedProperty, URIRef(foo+'lat')))
    g.add((URIRef(foo+'lat'), lat.lat, Literal(row['lat'], datatype=XSD.float)))
     
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'long')))
    g.add((URIRef(foo+'long'), long.long, Literal(row['long'], datatype=XSD.float)))
    
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'Temperature')))
    g.add((URIRef(foo+'Temperature'), foo.hasValue, Literal(row['Temperature'], datatype=XSD.double)))
    
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'Speed')))
    g.add((URIRef(foo+'Speed'), foo.hasValue, Literal(row['Speed'], datatype=XSD.float)))
    
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'Altitude')))
    g.add((URIRef(foo+'Altitude'),  alt.alt, Literal(row['Altitude'], datatype=XSD.float)))
   
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'Direction')))
    g.add(( URIRef(foo+'Direction'), foo.hasValue, Literal(row['Direction'], datatype=XSD.float)))
    
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'Distance')))
    g.add((URIRef(foo+'Distance'), foo.hasValue, Literal(row['Distance'], datatype=XSD.float)))
    
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'HDOP')))
    g.add((URIRef(foo+'HDOP'), foo.hasValue, Literal(row['HDOP'], datatype=XSD.integer)))
 
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'Cov')))
    g.add((URIRef(foo+'Cov'), foo.hasValue, Literal(row['Cov'], datatype=XSD.integer)))
    
    g.add((URIRef(foo+row['ID']), sosa.observedProperty, URIRef(foo+'Count')))
    g.add((URIRef(foo+'Count'), foo.hasValue, Literal(row['Count'], datatype=XSD.integer)))
    
    g.add((URIRef(foo+row['ID']), sosa.hasFeatureOfInterest, URIRef(foo+'ElephasMaximus')))
    
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Class.Mammalia))
    
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Family.Elephantidae))
  
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Kingdom.Animalia))
          
    g.add((URIRef(foo+'ElephasMaximus'), RDF.type, Phylum.Chordata))

In [8]:
print(g.serialize(format='turtle'))

@prefix ns1: <http://www.w3.org/ns/sosa/> .
@prefix ns2: <http://www.ontology/ns/foo/1.1#> .
@prefix ns3: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ns4: <http://schema.org/dgfc/elephant#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns2:100SAT32 a ns1:Observation ;
    ns4:Jasmin "100SAT32"^^xsd:string ;
    ns1:hasFeatureOfInterest ns2:ElephasMaximus ;
    ns1:hasResult "100SAT32"^^xsd:string ;
    ns1:madeBySensor <http://schema.org/DGFC/elephant#Jasmin> ;
    ns1:observedProperty ns2:Altitude,
        ns2:Count,
        ns2:Cov,
        ns2:Direction,
        ns2:Distance,
        ns2:GMTDate,
        ns2:HDOP,
        ns2:LocalDate,
        ns2:LocalTime,
        ns2:Speed,
        ns2:Temperature,
        ns2:lat,
        ns2:long .

ns2:101SAT32 a ns1:Observation ;
    ns4:Jasmin "101SAT32"^^xsd:string ;
    ns1:hasFeatureOfInterest ns2:ElephasMaximus ;
    ns1:hasResult "101SAT32"^^xsd:string ;
    ns1:madeBySensor <http://schema.org/DGFC/elephant#Jasmin> ;
 

In [10]:
# saving ontology to disk
g.serialize("Jasminnew13March9.rdf", format="ttl")

<Graph identifier=N740fcac7005546ba8725871561230db1 (<class 'rdflib.graph.Graph'>)>